# ERG Data Challenge Week 2
## Plotting NDVI

In [1]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm, trange
import tqdm
from pprint import pprint 
import statsmodels.api as sm
import math

from IPython.display import Image

import folium
from folium import plugins
import importlib

# import geopandas as gpd
from PIL import Image

### Start Here!

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWj7kl1mkbe9MCLFHYgidKB-FUq-nRDFOq6ff7Houm8depxucETS6s4

Successfully saved authorization token.


In [3]:
ee.Initialize()

### Run these cells to allow us to use these functions

In [4]:
"""
Standard code for adding an EE object to a folium map. copied almost directly from EE Python API 
"""
def add_ee_layer2(self, eeImageObject, visParams, name):
    map_id_dict = ee.Image(eeImageObject).getMapId(visParams)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)
folium.Map.add_ee_layer2 = add_ee_layer2

In [5]:
"""
Whenever you visualize something on EE using folium you need to name the band, and as a result,
we have different vis params for whether we are visualizing a single image or a median composite etc 
"""
vis_params_red = {
  'min': 0,
  'max': 6000,
  'palette': ['000000', 'FF0000']}
vis_params_green = {
  'min': 0,
  'max': 6000,
  'palette': ['000000', '00FF00']}
vis_params_blue = {
  'min': 0,
  'max': 6000,
  'palette': ['000000', '0000FF']}

# Set visualization parameters for multiple bands (cannot provide palette)
vis_params_median = {
  'min': 0,
  'max': 4000,
'bands': ['B4_median', 'B3_median', 'B2_median', 'B8_median']}
vis_params_median_single = {
  'min': 0,
  'max': 1,
'bands': ['B8_median']}
vis_params_multi = {
    'min': 0,
  'max': 4000,
'bands': ['B4', 'B3', 'B2']}
vis_params_B = {
    'min': 0,
  'max': 4000,
'bands': ['B']}
vis_params_Bsum = {
    'min': 0,
  'max': 4000,
'bands': ['B_sum']}

In [6]:
# This is where we put in Chicago's latitude and longitude:
# Example: lbx = lower bound x
# Note: box is 0.1 x 0.1 lat long units
lbx = -87.6298
ubx = lbx + 0.1
lby = 41.8
uby = lby + 0.1

# Make an Earth Engine object polygon with the above coordinates 
poly = ee.Geometry.Polygon(
    [[[lbx, lby],
      [lbx, uby],
      [ubx, uby],
      [ubx, lby]]])

# Alternatively our point of interest:
# This defines the lower left hand corner of our rectangle
poi = ee.Geometry.Point(lbx,lby)

# Filetering by year could be set up:
# This is NOT yet functional
month = 8
start_year = 2014
end_year = 2018

Import Sentinel Data from Google Earth Engine:}

In [7]:
col_sent = ee.ImageCollection("MODIS/061/MOD11A1")

Filter clouds and choose least cloudy images:

In [8]:
# Subset images with less than 5% cloud cover, then subset those which intersect the 
# lower left corner of the rectangle 'poly' we've just defined above.
clouds = col_sent.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 
                                      5)).filterBounds(poly)

# Adjust chosen bands to anything you want, and the featurization will flow from it
chosen_bands = ['B2', 'B3', 'B4', 'B8']

# choose only desired bands from the image 
bands = clouds.select(chosen_bands)
s = bands.sort('CLOUD_COVER')
# s contains all sentinel images which satisfied the above two filters 

# Here you  choose whether you want to have:
#     (1) the least cloudy image s.first()
#     (2) the median of all images in s 


# Option 1:
#scene = s.first()

# Option 2:
scene = s.reduce(ee.Reducer.median())

#### Useful docs:

Earth Engine API: 
(In other words, how to manipulate earth engine objects) \
https://developers.google.com/earth-engine/apidocs/ee-array-add

Sentinel Data Specifications:
(What what measured and what is it called) \
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

In [9]:
# Applying the same filters as above, but only to the Near Infared Band (NIR):
nir = clouds.select(['B8']).reduce(ee.Reducer.median())

# Do the same for the red band
red = clouds.select(['B4']).reduce(ee.Reducer.median())

$$ NDVI = \frac{NIR + Red}{NIR - Red}$$

In [12]:
# Calculate NDVI (Normalized Difference Vegetation Index):
numer = nir.add(red)
denom = nir.subtract(red)

ndvi = denom.divide(numer)

In [13]:
fmap = folium.Map(location=[lby, lbx], zoom_start=10)
fmap.add_ee_layer2(ndvi, vis_params_median_single, 'Scene 1')
fmap